# Port of Rotterdam - OpenTNSim

In [1]:
import opentnsim

from importlib import reload 
opentnsim = reload(opentnsim) 

In [2]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib
import io
import urllib
import tempfile
import functools
import logging
import pickle

# package(s) related to the simulation
import simpy
import networkx as nx  

# spatial libraries 
import shapely.geometry
import shapely.wkt
import pyproj
import shapely.geometry
from osgeo import ogr, osr
from simplekml import Kml, Style
import folium
import yaml

# package(s) for data handling
import requests
import math             
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# define the coorinate system
geod = pyproj.Geod(ellps="WGS84")

In [3]:
# Link to the latest version of the 'Vaarweginformatie.nl' network
#url = 'https://zenodo.org/record/3981105/files/network_digital_twin_v0.1.yaml'
url = 'https://zenodo.org/record/4578289/files/network_digital_twin_v0.2.yaml'

In [4]:
# Link to vessel database
location_vessel_database = "Vessels/Vessel-database-2.csv"

## 0. Load useful methods

### A) Network functions
I think Fedor included working with the FIS in another version of OpenTNSim (for the P&W exercise) -- need to check

In [5]:
@functools.lru_cache
def load_fis_network(url):
    """load the topological fairway information system network (vaarweginformatie.nl)"""

    # get the data from the url
    resp = requests.get(url)
    # convert to file object
    stream = io.StringIO(resp.text)
    
    # This will take a minute or two
    # Here we convert the network to a networkx object
    G = nx.read_yaml(stream)
    
#     with open(url, 'r') as fh:
#         G = yaml.load(fh, Loader=yaml.Loader)

    # some brief info
    n_bytes = len(resp.content)
    msg = '''Loaded network from {url} file size {mb:.2f}MB. Network has {n_nodes} nodes and {n_edges} edges.'''
    summary = msg.format(url=url, mb=n_bytes / 1000**2, n_edges=len(G.edges), n_nodes=len(G.nodes))
    logger.info(summary)

    # The topological network contains information about the original geometry. 
    # Let's convert those into python shapely objects for easier use later
    for n in G.nodes:
        G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
    for e in G.edges:
        edge = G.edges[e]
        edge['geometry'] = shapely.wkt.loads(edge['Wkt'])
        edge['length'] = edge_length(edge)    
    
    return G 

In [6]:
def find_closest_node(G, point):
    """find the closest node on the graph from a given point"""
    
    distance = np.full((len(G.nodes)), fill_value=np.nan)
    for ii, n in enumerate(G.nodes):
        distance[ii] = point.distance(G.nodes[n]['geometry'])
    name_node = list(G.nodes)[np.argmin(distance)]
    distance_node = np.min(distance)
    
    return name_node, distance_node

In [7]:
def find_closest_edge(G, point):
    """find the closest edge on the graph from a given point"""
    
    distance = np.full((len(G.edges)), fill_value=np.nan)
    for ii, e in enumerate(G.edges):
        distance[ii] = point.distance(G.edges[e]['geometry'])
    name_edge = list(G.edges)[np.argmin(distance)]
    distance_edge = np.min(distance)
    
    return name_edge, distance_edge

In [8]:
def edge_length(edge):
    """compute the great circle length of an edge
    The network version 0.1 contains the lat/lon distance in a length property. 
    But we need the "great circle" or projected distance. 
    Let's define a function to recompute it.
    """
    
    # get the geometry
    geom = edge['geometry']
    # get lon, lat
    lats, lons = np.array(geom).T
    # this requires pyproj 2.3.0
    distance = geod.line_length(lons, lats)

    return distance

### 1. Load Vaarweginformatie.nl graph (saved with PoR features)

In [9]:
# create a cached version to speed up loading (remove cached file if a better yaml file is available)
fname = "fis_wdata/FIS_wdata.pkl"
#fname = "fis_cache/FIS.pkl"

if os.path.exists(fname):
    print('I am loading cached network')
    with open(fname, 'rb') as pkl_file:
        FG = pickle.load(pkl_file)
        pkl_file.close()

else:
    print('I am getting new network')
    FG = load_fis_network(url)

    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, 'wb') as pkl_file:
        pickle.dump(FG, pkl_file)
        pkl_file.close()

I am loading cached network


#### 1.3 Plot the entire network

In [10]:
# Plot the network 
m = folium.Map(location=[51.9, 4.4], zoom_start = 11, tiles="cartodbpositron")

for edge in FG.edges(data = True):
    points_x = list(edge[2]["geometry"].coords.xy[0])
    points_y = list(edge[2]["geometry"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
        
    folium.PolyLine(line, weight = 2, popup = (edge[2]["StartJunctionId"], edge[2]["EndJunctionId"])).add_to(m)

m

### 2. Prepare vessel objects

In [11]:
sim_start_time = datetime.datetime(2019, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)
sim_start_time

datetime.datetime(2019, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)

#### 2.1 Load vessel database

In [12]:
# import the route information
df_trips = pd.read_parquet('03_vessels/df_3epet.parquet')
df_trips = df_trips[df_trips['vessellength']>150]

In [13]:
for i, path in enumerate(df_trips['path']): 
    newpath = []
    for node in path: 
        try: 
            newpath.append(float(node))
        except: 
            newpath.append(node)
    df_trips.at[df_trips.index[i], 'path'] = newpath

In [14]:
for i, speed in enumerate(df_trips['speed']):
    df_trips.at[df_trips.index[i], 'speed'] = list(speed)

In [15]:
# Example calculations UKC and FWA over route
iship = df_trips.index[0]
path = df_trips.loc[iship, 'path']
T_s = df_trips.loc[iship, 'draughtMarine']
direction = 'inbound'

for i in range(1,len(path)):
    edge = path[i-1], path[i]
    
    if FG.has_edge(edge[0], edge[1]):
        # FWA
        FWA = T_s * FG.edges[edge]['FWA']
        
        # UKC
        if T_s < 17.4: 
            UKC_a = FG.edges[edge]['UKC_a (T<17.4)']
        else: 
            UKC_a = FG.edges[edge]['UKC_a (T>=17.4)']
            
        if direction == 'outbound':
            UKC_b = FG.edges[edge]['UKC_b (outbound)']
        else: 
            UKC_b = FG.edges[edge]['UKC_b (outbound)']
        UKC = UKC_a + UKC_b*T_s
        
    else: 
        FWA = float('nan')
        UKC = float('nan')
    print('edge {}: UKC = {}, FWA = {}'.format(edge, UKC, FWA))

edge (8864185.0, 8866260.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8866260.0, 8864288.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8864288.0, 8861022.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8861022.0, 8860701.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8860701.0, 8864748.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8864748.0, 8865822.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8865822.0, 8866305.0): UKC = 1.0, FWA = 0.09800000190734863
edge (8866305.0, 8864266.0): UKC = 0.9800000190734863, FWA = 0.09800000190734863
edge (8864266.0, 8862925.0): UKC = 0.9800000190734863, FWA = 0.09800000190734863
edge (8862925.0, 8864465.0): UKC = 0.9800000190734863, FWA = 0.09800000190734863
edge (8864465.0, 'S14716_B'): UKC = 0.9800000190734863, FWA = 0.09800000190734863
edge ('S14716_B', 'S14716_A'): UKC = 0.9800000190734863, FWA = 0.09800000190734863
edge ('S14716_A', 8860845.0): UKC = 0.9800000190734863, FWA = 0.09800000190734863
edge (8860845.0, 8861674.0): UKC = 0.98000001907

#### 2.2 Create multiple vessel objects with the new mixins, and use this in the vessel generator

In [19]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (opentnsim.core.Identifiable, opentnsim.core.ContainerDependentMovable, 
                          opentnsim.core.HasResource, opentnsim.core.Routeable, 
                          opentnsim.core.VesselProperties, opentnsim.energy_consumption_module.ConsumesEnergy), {})


In [20]:
# For testing purposes: default v = v_empty
def compute_v_provider(v_empty, v_full):
    return lambda x: v_empty

In [21]:
df_trips.drop(index=4173, inplace=True)

In [22]:
# identify which vessel type to extract from the vessel database (change the name to change the vessel used)
vessels = []

for i_vessel in df_trips.index:

    # path and route info
    path = df_trips['path'].loc[i_vessel]
    v_list = df_trips['speed'].loc[i_vessel]

    if len(path) - len(v_list) != 1: 
        print('Path and velocities input length incorrect (difference = {} and should be 1)'.format(len(path) - len(v_list)))

    start_point = path[0]

    # prepare data for the vessel
    data_vessel = {"env": None,
                   "name": df_trips['shipname'].loc[i_vessel],
                   "route": df_trips['path'].loc[i_vessel],
                   "geometry": FG.nodes[start_point]['geometry'],
                   "capacity": 1000,
                   "compute_v": compute_v_provider(v_empty=v_list, v_full=v_list),
                   "type": df_trips['vesseltypeERI'].loc[i_vessel],
                   "B": df_trips['vesselwidth'].loc[i_vessel],
                   "L": df_trips['vessellength'].loc[i_vessel],
                   "T_e": df_trips['draughtMarine'].loc[i_vessel],
                   "T_f": df_trips['draughtMarine'].loc[i_vessel],
                   "H_e": 10,
                   "H_f": 5,
                   "P_installed": 500,
                   "L_w": 1,
                   "C_b": 0.85
                  }
    
    vessel = TransportResource(**data_vessel)
    vessels.append(vessel)


The construction year of the engine is 2004
The construction year of the engine is 2010
The construction year of the engine is 1994
The construction year of the engine is 2002
The construction year of the engine is 2018
The construction year of the engine is 1963
The construction year of the engine is 2020
The construction year of the engine is 2006
The construction year of the engine is 2013
The construction year of the engine is 1974
The construction year of the engine is 2012
The construction year of the engine is 2009
The construction year of the engine is 1999
The construction year of the engine is 2011
The construction year of the engine is 1986
The construction year of the engine is 1997
The construction year of the engine is 1994
The construction year of the engine is 1998
The construction year of the engine is 2004
The construction year of the engine is 1977
The construction year of the engine is 2018
The construction year of the engine is 2011
The construction year of the eng

# 3. Create environment

In [ ]:
# sim.add_vessels(origin,destination,[],vessel)

In [ ]:
# # specify start time and duration
# simulation_start = sim_start_time 
# duration = 5*12.5*60*60 #seconds

# # create simulation with environment
# sim = model.Simulation(simulation_start,FG)
# env = sim.environment




# sim.run(duration = duration)

In [26]:
# Start simpy environment (at a given date and time) (simulation_start = datetime.datetime.now() to just start now)
simulation_start = sim_start_time # <============ start time
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())

# Add graph to environment
env.FG = FG 

In [33]:
env

# 4. Run simulation

In [ ]:
sim.add_vessels(origin,destination,[],vessel)

In [ ]:
simulation_start = datetime.datetime.now()
sim = model.Simulation(simulation_start,FG)
env = sim.environment
duration = 5*12.5*60*60 #seconds

sim.run(duration = duration)

In [27]:
def start(env, vessel):
    """The start process moves a vessel over the network 
    (from its 'vessel.geometry' position to the end of 'vessel.route')"""
    vessel.log_entry("Start sailing", env.now, "", vessel.geometry)
    yield from vessel.move()
    vessel.log_entry("Stop sailing", env.now, "", vessel.geometry)
        

In [28]:
for i, vessel in enumerate(vessels):
    
    # Add graph to environment
    vessel.env = env
    
    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))

env.run()

origin = 8868083.0, destination = 8863475.0
inode = 0, speed = 7.857539643698429, name = testschip-5953
origin = 8860614.0, destination = B5729_A
inode = 0, speed = 4.9633746885987735, name = testschip-6735
origin = 8866859.0, destination = 8866999.0
inode = 0, speed = 9.848134545748662, name = testschip-6735
origin = 8864283.0, destination = 8860614.0
inode = 0, speed = 5.591261482718618, name = testschip-6735
origin = 8866859.0, destination = 8866999.0
inode = 0, speed = 9.235068006296736, name = testschip-6735
origin = 8860614.0, destination = B5729_A
inode = 0, speed = 9.183999269507456, name = testschip-6735
origin = 8866859.0, destination = 8866999.0
inode = 0, speed = 12.6913682245053, name = testschip-6735
origin = 8867980.0, destination = 8867547.0
inode = 0, speed = 11.62633898884754, name = testschip-6457
origin = 8863253.0, destination = 8868083.0
inode = 0, speed = 7.770368464718986, name = testschip-2923
origin = 8866305.0, destination = 8864266.0
inode = 0, speed = 21.01

inode = 1, speed = 13.181707046213566, name = testschip-4951
origin = 8861732.0, destination = 8868083.0
inode = 1, speed = 6.0105151915236235, name = testschip-9086
inode = 1, speed = 10.709080921901805, name = testschip-12115
origin = 8860614.0, destination = B5729_A
inode = 1, speed = 8.771800775328975, name = testschip-6735
origin = 8867980.0, destination = 8867547.0
inode = 1, speed = 12.599318113162456, name = testschip-12115
inode = 2, speed = 21.07407337073666, name = testschip-6457
origin = 8862214.0, destination = 8866780.0
inode = 4, speed = 19.949395117852728, name = testschip-10756
inode = 0, speed = 12.6913682245053, name = testschip-6735
inode = 1, speed = 13.181707046213566, name = testschip-4951
origin = 8864266.0, destination = 8862925.0
inode = 1, speed = 12.117920595549108, name = testschip-9890
origin = 8863475.0, destination = 8865217.0
inode = 1, speed = 8.41947640901091, name = testschip-7363
origin = 8866999.0, destination = 8867980.0
inode = 1, speed = 15.1158

inode = 3, speed = 24.183479641015552, name = testschip-8961
inode = 2, speed = 21.07407337073666, name = testschip-6457
inode = 3, speed = 24.183479641015552, name = testschip-8961
origin = 8862925.0, destination = 8864465.0
inode = 2, speed = 23.538411030266147, name = testschip-9317
inode = 1, speed = 17.227825149390696, name = testschip-9735
inode = 1, speed = 16.947142226774695, name = testschip-11583
inode = 1, speed = 15.895866888494757, name = testschip-11372
inode = 1, speed = 12.629315404453287, name = testschip-6735
origin = 8864465.0, destination = S14716_B
inode = 3, speed = 22.45235274490734, name = testschip-9317
inode = 0, speed = 5.7422761347386055, name = testschip-5316
origin = 8868083.0, destination = 8863475.0
inode = 1, speed = 9.165162475440592, name = testschip-2923
inode = 1, speed = 12.602276888581814, name = testschip-9317
origin = 8867980.0, destination = 8867547.0
inode = 2, speed = 6.765001563420722, name = testschip-11583
inode = 3, speed = 22.45235274490

inode = 2, speed = 17.46142769462671, name = testschip-9735
inode = 1, speed = 12.03759506457212, name = testschip-8946
inode = 4, speed = 18.363285696240386, name = testschip-11583
inode = 3, speed = 20.93761037583957, name = testschip-12115
inode = 1, speed = 10.709080921901805, name = testschip-12115
origin = 8862925.0, destination = 8864465.0
inode = 2, speed = 17.317255084029206, name = testschip-11583
origin = 8864465.0, destination = S14716_B
inode = 3, speed = 13.392873303849273, name = testschip-9088
origin = 8864465.0, destination = S14716_B
inode = 3, speed = 17.4745848827113, name = testschip-8397
inode = 4, speed = 25.658053157814923, name = testschip-6735
origin = 8860596.0, destination = 8867980.0
inode = 3, speed = 8.081781205958238, name = testschip-7363
inode = 3, speed = 19.052248025849387, name = testschip-9265
origin = 8864465.0, destination = S14716_B
inode = 3, speed = 19.51705466899788, name = testschip-9735
origin = 8867449.0, destination = 8862332.0
inode = 3,

inode = 2, speed = 9.903409773713694, name = testschip-8961
inode = 4, speed = 17.125156285382776, name = testschip-8332
origin = 8862332.0, destination = 8862930.0
inode = 4, speed = 16.49416085783862, name = testschip-6735
inode = 4, speed = 19.23495777659265, name = testschip-6394
inode = 5, speed = 24.849849869184926, name = testschip-8110
inode = 3, speed = 15.853004199556443, name = testschip-9465
inode = 3, speed = 15.401809194936483, name = testschip-9265
inode = 3, speed = 19.51705466899788, name = testschip-9735
inode = 4, speed = 22.406820375250835, name = testschip-7363
inode = 2, speed = 13.071937174821501, name = testschip-6735
origin = 8861158.0, destination = 8861674.0
inode = 5, speed = 24.325831470924964, name = testschip-6120
origin = 8861158.0, destination = 8861674.0
inode = 6, speed = 22.932425636804442, name = testschip-5637
inode = 3, speed = 21.67923750567213, name = testschip-11372
inode = 3, speed = 17.4745848827113, name = testschip-8397
inode = 3, speed = 2

...

inode = 18, speed = 21.30018409488337, name = testschip-6735
inode = 7, speed = 7.9240417365996905, name = testschip-9086
inode = 7, speed = 11.000855191659863, name = testschip-9317
inode = 7, speed = 11.417730857197716, name = testschip-7363
inode = 7, speed = 9.787923539218351, name = testschip-9465
inode = 6, speed = 9.446568534582418, name = testschip-12115
inode = 7, speed = 6.7699110323186265, name = testschip-9735
origin = 8864465.0, destination = 8862925.0
inode = 14, speed = 18.140511563356934, name = testschip-9735
inode = 7, speed = 11.169628271998436, name = testschip-9317
inode = 7, speed = 9.327798447298925, name = testschip-8961
origin = 8862925.0, destination = 8864266.0
inode = 15, speed = 19.93761588829633, name = testschip-9735
origin = 8864465.0, destination = 8862925.0
inode = 8, speed = 13.60419699753151, name = testschip-9317
inode = 15, speed = 10.918448207251155, name = testschip-8385
origin = 8861158.0, destination = 8867547.0
inode = 8, speed = 10.8843655038

inode = 8, speed = 6.057447962319852, name = testschip-7363
inode = 7, speed = 9.310317681380196, name = testschip-12927
origin = 8867547.0, destination = 8867980.0
inode = 9, speed = 6.3966372767152775, name = testschip-5000
origin = 8867980.0, destination = 8860596.0
inode = 10, speed = 8.84644510002893, name = testschip-8961
inode = 7, speed = 8.808005632611644, name = testschip-12440
inode = 7, speed = 8.15578141960497, name = testschip-6043
inode = 15, speed = 4.8723536975454405, name = testschip-10756
inode = 7, speed = 9.530205036566251, name = testschip-9088
inode = 7, speed = 9.787923539218351, name = testschip-9465
inode = 21, speed = 25.365989294767964, name = testschip-6735
inode = 12, speed = 13.286977263732334, name = testschip-5953
inode = 7, speed = 10.879379740861822, name = testschip-4951
inode = 10, speed = 2.647804062383816, name = testschip-9317
inode = 14, speed = 20.99332238335375, name = testschip-5000
inode = 6, speed = 8.027232952748912, name = testschip-8946


inode = 7, speed = 8.791072304121702, name = testschip-12115
inode = 24, speed = 18.40403513510697, name = testschip-6735
inode = 7, speed = 6.448217629147706, name = testschip-8946
inode = 24, speed = 12.612487118856555, name = testschip-6735
origin = 8861158.0, destination = 8867547.0
inode = 8, speed = 6.146540190227468, name = testschip-9265
origin = 8860596.0, destination = 8865217.0
inode = 11, speed = 8.217675051844095, name = testschip-6043
inode = 24, speed = 18.40403513510697, name = testschip-6735
inode = 7, speed = 7.481511946677999, name = testschip-11583
inode = 7, speed = 6.7699110323186265, name = testschip-9735
inode = 7, speed = 7.9240417365996905, name = testschip-9086
inode = 21, speed = 15.432341667093821, name = testschip-6735
origin = 8867547.0, destination = 8867980.0
inode = 9, speed = 4.575257057141874, name = testschip-8397
inode = 7, speed = 6.389915666200359, name = testschip-10445
inode = 24, speed = 12.612487118856555, name = testschip-6735
origin = 88611

inode = 27, speed = 8.701747302976008, name = testschip-6735
inode = 2, speed = 0.9320641033316714, name = testschip-8385
origin = 8867547.0, destination = 8867980.0
inode = 9, speed = 5.373019504020754, name = testschip-10445
origin = 8867980.0, destination = 8866999.0
inode = 28, speed = 3.941226440008277, name = testschip-6735
inode = 22, speed = 11.00450569497475, name = testschip-6735
inode = 7, speed = 2.671364297457504, name = testschip-12011
origin = 8867980.0, destination = 8860596.0
inode = 10, speed = 6.840943973661301, name = testschip-10445
inode = 7, speed = 6.448217629147706, name = testschip-8946
inode = 22, speed = 11.00450569497475, name = testschip-6735
inode = 28, speed = 3.941226440008277, name = testschip-6735
inode = 16, speed = 5.952931268385742, name = testschip-8385
inode = 25, speed = 8.911838115350195, name = testschip-6735
origin = 8867547.0, destination = 8867980.0
inode = 9, speed = 4.427918070123839, name = testschip-9890
origin = 8861158.0, destination 

inode = 24, speed = 1.1534552692261897, name = testschip-8385
inode = 13, speed = 7.7483820012313185, name = testschip-8385
inode = 13, speed = 7.7483820012313185, name = testschip-8385
inode = 13, speed = 7.7483820012313185, name = testschip-8385
inode = 13, speed = 7.7483820012313185, name = testschip-8385
inode = 24, speed = 1.1534552692261897, name = testschip-8385
inode = 13, speed = 7.7483820012313185, name = testschip-8385
inode = 24, speed = 1.1534552692261897, name = testschip-8385
inode = 13, speed = 7.7483820012313185, name = testschip-8385
origin = 8860845.0, destination = S14716_A
inode = 14, speed = 9.649557500573364, name = testschip-8385
inode = 14, speed = 9.649557500573364, name = testschip-8385
origin = S14716_A, destination = S14716_B
inode = 15, speed = 10.08910456284078, name = testschip-8385
origin = S14716_B, destination = 8864465.0
inode = 16, speed = 10.280091038994325, name = testschip-8385
inode = 16, speed = 10.280091038994325, name = testschip-8385
origin 

# 5. Evaluate output

In [29]:
# basic inspection of the first vessel of the dataset
df_log = pd.DataFrame.from_dict(vessel.log)

In [32]:
df_log

,Message,Timestamp,Value,Geometry
0,Start sailing,2019-01-01 00:00:00.000000,,POINT (4.28248423401617 51.8883798397893)
1,Sailing from node 8863090.0 to node 8863475.0 ...,2019-01-01 00:00:00.000000,testschip-12440,POINT (4.28248423401617 51.8883798397893)
2,Sailing from node 8863090.0 to node 8863475.0 ...,2019-01-01 00:01:28.460093,testschip-12440,POINT (4.28442105542451 51.8846316430902)
3,Sailing from node 8863475.0 to node 8865217.0 ...,2019-01-01 00:01:28.460093,testschip-12440,POINT (4.28441264524576 51.8846350350828)
4,Sailing from node 8863475.0 to node 8865217.0 ...,2019-01-01 00:02:23.645046,testschip-12440,POINT (4.29044058645646 51.8858419622723)
...,...,...,...,...
83,Sailing from node 8862925.0 to node 8864266.0 ...,2019-01-01 00:20:41.010849,testschip-12440,POINT (4.10714114771394 51.9781909994644)
84,Sailing from node 8862925.0 to node 8864266.0 ...,2019-01-01 00:22:09.182713,testschip-12440,POINT (4.0841372668255 51.9838622952073)
85,Sailing from node 8862925.0 to node 8864266.0 ...,2019-01-01 00:22:09.182713,testschip-12440,POINT (4.0841372668255 51.9838622952073)
86,Sailing from node 8862925.0 to node 8864266.0 ...,2019-01-01 00:22:11.346430,testschip-12440,POINT (4.08355198540297 51.9839636810505)


In [30]:
# create new dataframe 

df_newlog = pd.DataFrame(columns=['From edge', 'To edge', 'Sub edge', 'Timestamp', 'From geometry', 'To geometry', 'Duration', 'Distance', 'Speed'])

for i in range(1,len(df_log.index)): 
    #print(df_log['Message'].loc[i])
    #if i<len(df_log)-1:
        
    # message 
    message = df_log['Message'].iloc[i].split()
    if message[-1] == 'start':

        # determine duration and distance
        duration = (df_log['Timestamp'].loc[i+1] - df_log['Timestamp'].loc[i]).total_seconds()
        distance = geod.inv(df_log['Geometry'].loc[i].x, df_log['Geometry'].loc[i].y, 
            df_log['Geometry'].loc[i+1].x, df_log['Geometry'].loc[i+1].y)[2]
        speed = distance / duration
            
        df_newlog = df_newlog.append({'From edge': message[3], 'To edge': message[6], 'Sub edge': message[9],
                                      'Timestamp': df_log['Timestamp'].loc[i], 
                                      'From geometry': df_log['Geometry'].loc[i], 'To geometry': df_log['Geometry'].loc[i+1],
                                      'Duration': duration, 'Distance': distance, 'Speed': speed}, ignore_index=True)

In [31]:
df_newlog

,From edge,To edge,Sub edge,Timestamp,From geometry,To geometry,Duration,Distance,Speed
0,8863090.0,8863475.0,0,2019-01-01 00:00:00.000000,POINT (4.28248423401617 51.8883798397893),POINT (4.28442105542451 51.8846316430902),88.460093,437.845445,4.949638
1,8863475.0,8865217.0,0,2019-01-01 00:01:28.460093,POINT (4.28441264524576 51.8846350350828),POINT (4.29044058645646 51.8858419622723),55.184953,436.228995,7.904854
2,8865217.0,8860596.0,0,2019-01-01 00:02:23.645046,POINT (4.29044058645646 51.8858419622723),POINT (4.30330688740968 51.8884241525286),113.493354,931.279021,8.205582
3,8860596.0,8867980.0,0,2019-01-01 00:04:17.138400,POINT (4.30330688740968 51.8884241525286),POINT (4.30901781752353 51.8895642784668),57.273236,413.142566,7.213536
4,8867980.0,8867547.0,0,2019-01-01 00:05:14.411636,POINT (4.30901781752353 51.8895642784668),POINT (4.30918191152922 51.8905995237284),18.770465,115.739513,6.166044
5,8867547.0,8867980.0,0,2019-01-01 00:05:33.182101,POINT (4.30918191152922 51.8905995237284),POINT (4.30901781752353 51.8895642784668),18.071505,115.739513,6.404531
6,8867980.0,8867547.0,0,2019-01-01 00:05:51.253606,POINT (4.30901781752353 51.8895642784668),POINT (4.30918191152922 51.8905995237284),17.422730,115.739513,6.643018
7,8867547.0,8861158.0,0,2019-01-01 00:06:08.676336,POINT (4.30918191152922 51.8905995237284),POINT (4.30992074255024 51.8951877409715),34.687151,513.036924,14.790402
8,8861158.0,8861674.0,0,2019-01-01 00:06:43.363487,POINT (4.30992837177677 51.8951847057235),POINT (4.30742197256351 51.8954034825434),9.174204,174.244548,18.992879
9,8861158.0,8861674.0,1,2019-01-01 00:06:52.537691,POINT (4.30742197256351 51.8954034825434),POINT (4.29874400442962 51.8965645780719),32.179216,611.175984,18.992880
